In [1]:
import pandas as pd
import numpy as np

In [2]:
audi = pd.read_csv('audi.csv')

In [3]:
audi.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [191]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB


In [145]:
audi.describe()

,year,price,mileage,tax,mpg,engineSize
count,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000
mean,2017.100675,22896.685039,24827.244001,126.011436,50.770022,1.930709
std,2.167494,11714.841888,23505.257205,67.170294,12.949782,0.602957
min,1997.000000,1490.000000,1.000000,0.000000,18.900000,0.000000
25%,2016.000000,15130.750000,5968.750000,125.000000,40.900000,1.500000
50%,2017.000000,20200.000000,19000.000000,145.000000,49.600000,2.000000
75%,2019.000000,27990.000000,36464.500000,145.000000,58.900000,2.000000
max,2020.000000,145000.000000,323000.000000,580.000000,188.300000,6.300000


In [47]:
audi.model.value_counts()

 A3     1929
 Q3     1417
 A4     1381
 A1     1347
 A5      882
 Q5      877
 Q2      822
 A6      748
 Q7      397
 TT      336
 A7      122
 A8      118
 Q8       69
 RS6      39
 RS3      33
 RS4      31
 RS5      29
 R8       28
 S3       18
 SQ5      16
 S4       12
 SQ7       8
 S8        4
 S5        3
 A2        1
 RS7       1
Name: model, dtype: int64

In [156]:
audi_dums = pd.get_dummies(audi, columns=['model', 'transmission', 'fuelType'], drop_first=True)

In [81]:
audi_dums.drop(columns=['model'], inplace=True)

In [174]:
# note: change to only scale X and not y (price)
rscaler = RobustScaler()
rscaled_audi = rscaler.fit_transform(audi_dums)
rscaled_audi = pd.DataFrame(rscaled_audi, columns=audi_dums.columns)

In [291]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

In [199]:
audi.dtypes['model'] == 'float'

False

In [202]:
set(audi['fuelType'].unique())

{'Diesel', 'Hybrid', 'Petrol'}

In [208]:
ex = [i for i in range(10) if i == -1]
if ex:
    print(3)
else:
    print(0)

0


In [844]:
def prepare(data, target, test_size=0.2, random_state=None, stratified=False, n_splits=1):
        data = data.copy()

        # organizing the features into categories
        # numericals = [col for col in data.columns if (data.dtypes[col] == 'int' or data.dtypes[col] == 'float') and set(data[col].unique()) != {0, 1}]
        categoricals = [col for col in data.columns if data.dtypes[col] != 'int' or set(data[col].unique()) == {0, 1}]
        # numbers = [col for col in data.columns if data.dtypes[col] == 'int' or data.dtypes[col] == 'float']
        non_numbers = [col for col in data.columns if data.dtypes[col] != 'int' and data.dtypes[col] != 'float']
        
        if target in categoricals:
            raise ValueError('Please enter a continuous variable as a target.')
        
        # impute missing data
        
        
        # creating dummies if needed
        if non_numbers:
            data = pd.get_dummies(data, columns=non_numbers, drop_first=True)
            
        # separating the predictors and the target
        predictors = list(data.columns)
        predictors.remove(target)
        
        # creating the train and test sets
        if stratified:
            # use pd.cut to split target into groups
            target_col = data[target]
            data['target_cat'] = pd.cut(target_col, 
                                        bins=[target_col.min(), target_col.quantile(0.25), target_col.median(), target_col.quantile(0.75), target_col.max()], 
                                        include_lowest=True)
            
            splitter = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
            for train_index, test_index in splitter.split(data, data['target_cat']):
                strat_train_set = data.loc[train_index]
                strat_test_set = data.loc[test_index]
                
            # for observation:
            print(data['target_cat'].value_counts() / len(data)) 
            print(strat_test_set['target_cat'].value_counts() / len(strat_test_set)) 
                
            # drop categorical feature for target
            for set_ in (strat_train_set, strat_test_set):
                set_.drop('target_cat', axis=1, inplace=True)
                
            # finish splitting sets
            X_train = strat_train_set[predictors]
            X_test = strat_test_set[predictors]
            y_train = pd.DataFrame(np.c_[strat_train_set[target]], columns=[target])
            y_test = pd.DataFrame(np.c_[strat_test_set[target]], columns=[target])
        else:
            X = data[predictors]
            y = pd.DataFrame(np.c_[data[target]], columns=[target])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
            

        return X_train, X_test, y_train, y_test

In [845]:
X_train, X_test, y_train, y_test = prepare(audi, 'price', random_state=3, stratified=False)

In [855]:
y_test

,price
4201,28000
10650,6995
7326,35490
10501,12999
6565,22400
...,...
5390,23766
5181,13141
4843,31490
2034,31998


In [438]:
import statsmodels.api as sm

X1 = sm.add_constant(X_train)
ols = sm.OLS(y_train, X1)
lr = ols.fit()

In [439]:
lr.summary2().tables[1]['P>|t|']

const                      0.000000e+00
year                       0.000000e+00
mileage                   6.006501e-144
tax                       5.431224e-205
mpg                        0.000000e+00
engineSize                8.573097e-216
model_ A2                  1.145682e-18
model_ A3                  3.309723e-17
model_ A4                  3.209425e-20
model_ A5                  1.335164e-48
model_ A6                  1.522958e-53
model_ A7                  4.997917e-28
model_ A8                  3.973269e-68
model_ Q2                  2.377118e-12
model_ Q3                  3.419262e-58
model_ Q5                 8.440137e-180
model_ Q7                  0.000000e+00
model_ Q8                  0.000000e+00
model_ R8                  0.000000e+00
model_ RS3                 6.430664e-45
model_ RS4                2.555538e-140
model_ RS5                4.643141e-119
model_ RS6                2.825508e-226
model_ RS7                 5.616247e-07
model_ S3                  3.920778e-04


In [440]:
selected_features = [col for col in X_train.columns if lr.summary2().tables[1]['P>|t|'][col] <= 0.05]
selected_features

['year',
 'mileage',
 'tax',
 'mpg',
 'engineSize',
 'model_ A2',
 'model_ A3',
 'model_ A4',
 'model_ A5',
 'model_ A6',
 'model_ A7',
 'model_ A8',
 'model_ Q2',
 'model_ Q3',
 'model_ Q5',
 'model_ Q7',
 'model_ Q8',
 'model_ R8',
 'model_ RS3',
 'model_ RS4',
 'model_ RS5',
 'model_ RS6',
 'model_ RS7',
 'model_ S3',
 'model_ S4',
 'model_ SQ5',
 'model_ SQ7',
 'model_ TT',
 'transmission_Manual',
 'fuelType_Hybrid',
 'fuelType_Petrol']

In [441]:
X_train[selected_features]

,year,mileage,tax,mpg,engineSize,model_ A2,model_ A3,model_ A4,model_ A5,model_ A6,...,model_ RS6,model_ RS7,model_ S3,model_ S4,model_ SQ5,model_ SQ7,model_ TT,transmission_Manual,fuelType_Hybrid,fuelType_Petrol
2601,2015,46784,145,52.3,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9532,2016,12900,0,67.3,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1541,2019,5304,145,40.9,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6226,2019,6857,145,51.4,1.6,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8544,2015,36196,0,80.7,1.6,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6400,2019,6159,145,52.3,1.6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9160,2017,25249,145,67.3,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
9859,2013,51535,20,68.9,1.6,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1688,2015,52539,30,55.4,1.4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [365]:
y_train

,price
2601,15998
9532,12490
1541,21790
6226,23950
8544,10000
...,...
6400,22150
9160,13490
9859,8299
1688,11250


In [851]:
knn_reg = LinearRegression()
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test)

0.8879353446048815

In [638]:
op_knn = OptimalKNRegressor(3)
op_knn.find(data=audi, target='price')

KNeighborsRegressor(n_neighbors=3, p=1)

In [639]:
op_knn.score()

0.9511411883559433

In [640]:
op_knn.best_params

{'n_neighbors': 3, 'p': 1}

In [755]:
op_tree = OptimalDTreeRegressor(3)
op_tree.find(data=audi, target='price')

DecisionTreeRegressor(max_depth=15, random_state=3)

In [756]:
op_tree.score()

0.94945655345777

In [759]:
s = str(op_tree.model)
s

'DecisionTreeRegressor(max_depth=15, random_state=3)'

In [643]:
op_lin = OptimalLinearRegression(3)
op_lin.find(data=audi, target='price')

LinearRegression()

In [644]:
op_lin.score()

0.8876961846248463

In [691]:
op_lin_b = OptimalLinearRegression(3)
op_lin_b.find(data=audi, target='price')

LinearRegression()

In [692]:
op_lin_b.score()

0.8876961846248463

In [649]:
op_lin2 = OptimalLinearRegression(3)
op_lin2.find(X_train, y_train)

LinearRegression()

In [650]:
best_lin2_features = op_lin2.best_features

In [651]:
op_lin2.score(X_test[best_lin2_features], y_test)

0.8876961846248463

In [688]:
op_svr = OptimalLinearSVR(3)
op_svr.find(data=audi, target='price')

LinearSVR(C=1000, epsilon=0.5, random_state=3)

In [689]:
op_svr.score()

0.8382862658830257

In [586]:
op_sgd = OptimalSGDRegressor(3)
#op_sgd.find(data=audi, target='price')

In [584]:
op_sgd.score()

ValueError: There is no model to use for scoring

In [657]:
op_forest = OptimalRFRegressor(3)
op_forest.find(data=audi, target='price')

RandomForestRegressor(min_samples_split=5, n_estimators=50, random_state=3)

In [658]:
op_forest.score()

0.9625363422388229

In [659]:
op_forest.best_params

{'min_samples_split': 5, 'n_estimators': 50}

In [567]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm

In [879]:
class BaseRegressor:
    
    def __repr__(self):
        return type(self).__name__ # finish implementation!!!
    

    def score(self, X_test=None, y_test=None):
        if not self.model:
            raise ValueError('There is no model to use for scoring')

        if X_test is not None and y_test is not None:
            return self.model.score(X_test, y_test)
        elif self.X_test is not None and self.y_test is not None:
            return self.model.score(self.X_test, self.y_test)
        else:
            raise ValueError('No test sets to use for scoring')
            

    def get_sets(self, save=False):
        X_train, X_test, y_train, y_test = self.X_train, self.X_test, self.y_train, self.y_test
        if not save:
            self.X_train, self.X_test, self.y_train, self.y_test = (None, None, None, None)
        return X_train, X_test, y_train, y_test
    
    
    
class OptimalDTreeRegressor(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            param_grid=[{'max_depth': [1, 2, 3, 4, 5, 10, 15, 20, 25, 50, 100, 200, None]}]
    ):
        if isinstance(data, pd.DataFrame) and isinstance(target, str):
            X_train, X_test, y_train, y_test = prepare(data, target, random_state=self.random_state)

            # save sets
            self.X_train = X_train
            self.X_test = X_test
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in either your data and target feature or training set and target')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train

        grid = GridSearchCV(DecisionTreeRegressor(), param_grid)
        grid.fit(self.X_train, self.y_train)

        optimal_depth = grid.best_params_['max_depth']

        self.model = DecisionTreeRegressor(max_depth=optimal_depth, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train)

        self.best_params = grid.best_params_

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)

        return self.model


class OptimalKNRegressor(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 10, 15, 20], 'p': [1, 2]}]
    ):
        if isinstance(data, pd.DataFrame) and isinstance(target, str):
            X_train, X_test, y_train, y_test = prepare(data, target, random_state=self.random_state)

            # standardize X_train and X_test
            robust_scaler = RobustScaler()
            X_train_scaled = robust_scaler.fit_transform(X_train)
            X_test_scaled = robust_scaler.fit_transform(X_test)
            
            # transform arrays back to DataFrame
            X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
            X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

            # save sets
            self.X_train = X_train_scaled
            self.X_test = X_test_scaled
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in either your data and target feature or training set and target')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train

        grid = GridSearchCV(KNeighborsRegressor(), param_grid)
        grid.fit(self.X_train, self.y_train)

        optimal_neighbors = grid.best_params_['n_neighbors']
        optimal_p = grid.best_params_['p']

        self.model = KNeighborsRegressor(n_neighbors=optimal_neighbors, p=optimal_p)
        self.model.fit(self.X_train, self.y_train)

        self.best_params = grid.best_params_

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)

        return self.model


class OptimalLinearRegression(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.selected_features = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            alpha=0.05
    ):
        if isinstance(data, pd.DataFrame) and isinstance(target, str):
            X_train, X_test, y_train, y_test = prepare(data, target, random_state=self.random_state)
            
            # selecting the features with p-values <= alpha
            X1_train = sm.add_constant(X_train)
            ols = sm.OLS(y_train, X1_train)
            lin_reg = ols.fit()

            selected_features = [col for col in X_train.columns if lin_reg.summary2().tables[1]['P>|t|'][col] <= alpha]

            # updating the sets to only have the selected features
            X_train = X_train[selected_features] 
            X_test = X_test[selected_features] # if X_test is not None else None (idk what this was for)

            # save sets
            self.X_train = X_train
            self.X_test = X_test
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in either your data and target feature or training set and target')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train

        self.model = LinearRegression()
        self.model.fit(self.X_train, self.y_train)

        self.selected_features = selected_features

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)
            # print('Warning: The training set has been updated. Please get the new selected features to use for further training and testing.')

        return self.model


class OptimalLinearSVR(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            param_grid=[{'C': [0.1, 1., 1.5, 3., 10., 50., 100., 1000.], 'epsilon': [0., 0.01, 0.1, 0.2, 0.3, 0.5]}]
    ):
        if isinstance(data, pd.DataFrame) and isinstance(target, str):
            X_train, X_test, y_train, y_test = prepare(data, target, random_state=self.random_state)

            # standardize X_train and X_test
            robust_scaler = RobustScaler()
            X_train_scaled = robust_scaler.fit_transform(X_train)
            X_test_scaled = robust_scaler.fit_transform(X_test)
            
            # transform arrays back to DataFrame
            X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
            X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

            # save sets
            self.X_train = X_train_scaled
            self.X_test = X_test_scaled
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in either your data and target feature or training set and target')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train
            
        grid = GridSearchCV(LinearSVR(), param_grid)
        grid.fit(self.X_train, self.y_train.values.ravel())

        optimal_C = grid.best_params_['C']
        optimal_epsilon = grid.best_params_['epsilon']

        self.model = LinearSVR(epsilon=optimal_epsilon, C=optimal_C, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train.values.ravel())

        self.best_params = grid.best_params_

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)

        return self.model

''' 
class OptimalSGDRegressor(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            param_grid = {'max_iter': [1, 5, 10, 100, 1000], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
    ):
        if isinstance(data, pd.DataFrame):
            X_train, X_test, y_train, y_test = cut(data, target, random_state=self.random_state)

            # standardize X_train and X_test
            robust_scaler = RobustScaler()
            X_train_scaled = robust_scaler.fit_transform(X_train)
            X_test_scaled = robust_scaler.fit_transform(X_test)

            # save sets
            self.X_train = X_train_scaled
            self.X_test = X_test_scaled
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in some data')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train
            
        grid = GridSearchCV(ElasticNet(), param_grid)
        grid.fit(self.X_train, self.y_train.values.ravel())

        optimal_iter = grid.best_params_['max_iter']
        optimal_alpha = grid.best_params_['alpha']
        #optimal_penalty = grid.best_params_['penalty']

        self.model = ElasticNet(max_iter=optimal_iter, alpha=optimal_alpha, tol=0.01, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train)

        self.best_params = grid.best_params_

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)

        return self.model    
'''
    

    # Ensemble Methods
class OptimalRFRegressor(BaseRegressor):
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None

    def find(
            self,
            X_train=None,
            y_train=None,
            data=None,
            target=None,
            param_grid=[{'n_estimators': [10, 50, 100], 'min_samples_split': [2, 5, 10, 25, 50]}]
    ):
        if isinstance(data, pd.DataFrame) and isinstance(target, str):
            X_train, X_test, y_train, y_test = prepare(data, target, random_state=self.random_state)

            # save sets
            self.X_train = X_train
            self.X_test = X_test
            self.y_train = y_train
            self.y_test = y_test
        elif X_train is None or y_train is None:
            raise ValueError('Please pass in either your data and target feature or training set and target')
        else:
            # temporarily save sets
            self.X_train = X_train
            self.y_train = y_train

        grid = GridSearchCV(RandomForestRegressor(), param_grid)
        grid.fit(self.X_train, self.y_train.values.ravel())

        optimal_split = grid.best_params_['min_samples_split']
        optimal_estimators = grid.best_params_['n_estimators']

        self.model = RandomForestRegressor(n_estimators=optimal_estimators, min_samples_split=optimal_split, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train.values.ravel())

        self.best_params = grid.best_params_

        if self.y_test is None:
            self.X_train, self.y_train = (None, None)

        return self.model


In [274]:
class BaseBoa:
    def hunt(self, data=None, X_train=None, X_test=None, y_train=None, y_test=None, target=None, digest=True):
        pass
    
    

In [880]:
class BaseBoa:
    
    NUM_REGRESSION_MODELS_SUPPORTED = 5
    
    reg_model_index = {0: 'dtree', 
                       1: 'knn', 
                       2: 'linreg', 
                       3: 'linsvr', 
                       4: 'rforest',
                       }
    
    def __repr__(self):
        return type(self).__name__ # finish implementation!!!
    
    
    def __len__(self):
        return len(self.ladder)

    
    def ladder(self):
        return self.ladder
    
    
    def optimal_model(self, rank=0):
        return self.ladder[rank][0]
    
    
    def model(self, rank=0):
        return self.ladder[rank][0].model
    
    
    def model_score(self, rank=0):
        return self.ladder[rank][1]
    
       
    def get_sets(self, rank=0, save=False):
        return self.ladder[rank][0].get_sets(save=save)
    


class RegressionBoa(BaseBoa):
    
    def __init__(self, random_state=None):
        self.random_state = random_state
        self.ladder = []
    
    
    def hunt(self, X_train=None, X_test=None, y_train=None, y_test=None, data=None, target=None):
        for i in range(BaseBoa.NUM_REGRESSION_MODELS_SUPPORTED):
            model_key = BaseBoa.reg_model_index[i]
            optimal_model = self.find_optimal(regressor=model_key, X_train=X_train, y_train=y_train, data=data, target=target)
            
            if X_test is not None and y_test is not None:
                score = optimal_model.score(X_test, y_test)
            else:
                score = optimal_model.score()
                
            self.ladder.append((optimal_model, score))
            
        self.ladder.sort(key=lambda tup: tup[1], reverse=True)
        
        return self.ladder
    
    
    def find_optimal(self, regressor, X_train=None, y_train=None, data=None, target=None):
        if regressor == 'dtree':
            model = self.find_optimal_dtree(X_train=X_train, y_train=y_train, data=data, target=target)
        elif regressor == 'knn':
            model = self.find_optimal_knn(X_train=X_train, y_train=y_train, data=data, target=target)
        elif regressor == 'linreg':
            model = self.find_optimal_linreg(X_train=X_train, y_train=y_train, data=data, target=target)
        elif regressor == 'linsvr':
            model = self.find_optimal_linsvr(X_train=X_train, y_train=y_train, data=data, target=target)
        elif regressor == 'rforest':
            model = self.find_optimal_rforest(X_train=X_train, y_train=y_train, data=data, target=target)
        else:
            raise ValueError('Requested regressor not found. Please choose either \'dtree\' or \'knn\' or \'linreg\' or \'linsvr\' or \'rforest\'')
            # change statement above
            
        return model
    
    
    def find_optimal_dtree(self, X_train=None, y_train=None, data=None, target=None):
        optimal_dtree = OptimalDTreeRegressor(self.random_state)
        optimal_dtree.find(X_train=X_train, y_train=y_train, data=data, target=target)
        
        return optimal_dtree
    
    
    def find_optimal_knn(self, X_train=None, y_train=None, data=None, target=None):
        optimal_knn = OptimalKNRegressor(self.random_state)
        optimal_knn.find(X_train=X_train, y_train=y_train, data=data, target=target)
        
        return optimal_knn
    
    
    def find_optimal_linreg(self, X_train=None, y_train=None, data=None, target=None):
        optimal_linreg = OptimalLinearRegression(self.random_state)
        optimal_linreg.find(X_train=X_train, y_train=y_train, data=data, target=target)
        
        return optimal_linreg
    
    
    def find_optimal_linsvr(self, X_train=None, y_train=None, data=None, target=None):
        optimal_linsvr = OptimalLinearSVR(self.random_state)
        optimal_linsvr.find(X_train=X_train, y_train=y_train, data=data, target=target)
        
        return optimal_linsvr
    
    
    def find_optimal_rforest(self, X_train=None, y_train=None, data=None, target=None):
        optimal_rforest = OptimalRFRegressor(self.random_state)
        optimal_rforest.find(X_train=X_train, y_train=y_train, data=data, target=target)
        
        return optimal_rforest
    
    
    

In [881]:
rboa = RegressionBoa(3)
rboa.hunt(data=audi, target='price')

[(OptimalRFRegressor, 0.9626985917160836),
 (OptimalKNRegressor, 0.9511411883559433),
 (OptimalDTreeRegressor, 0.94945655345777),
 (OptimalLinearRegression, 0.8876961846248463),
 (OptimalLinearSVR, 0.8383624775592265)]

In [882]:
rboa.ladder

[(OptimalRFRegressor, 0.9626985917160836),
 (OptimalKNRegressor, 0.9511411883559433),
 (OptimalDTreeRegressor, 0.94945655345777),
 (OptimalLinearRegression, 0.8876961846248463),
 (OptimalLinearSVR, 0.8383624775592265)]

In [883]:
rboa.model()

RandomForestRegressor(min_samples_split=5, random_state=3)

In [884]:
rboa.model_score(1)

0.9511411883559433

In [877]:
X_train, X_test, y_train, y_test = rboa.get_sets(4)

In [878]:
X_train.head()

,year,mileage,tax,mpg,engineSize,model_ A2,model_ A3,model_ A4,model_ A5,model_ A6,...,model_ S4,model_ S5,model_ S8,model_ SQ5,model_ SQ7,model_ TT,transmission_Manual,transmission_Semi-Auto,fuelType_Hybrid,fuelType_Petrol
0,-0.666667,0.913804,0.00,0.150000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.333333,-0.198624,-7.25,0.983333,-2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.666667,-0.448005,0.00,-0.483333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.666667,-0.397019,0.00,0.100000,-0.8,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.666667,0.566194,-7.25,1.727778,-0.8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [868]:
str(rboa.ladder[0][0])

'OptimalRFRegressor'

In [683]:
len([])

0

In [666]:
if None not in (2, None, 4):
    print(3)

In [233]:
knn = KNeighborsRegressor(n_neighbors=3)

In [234]:
knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [235]:
knn.score(X_test, y_test)

0.9491092778381446

In [250]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 3,
 'p': 2,
 'weights': 'uniform'}

In [282]:
b = None
if not b:
    print(3)
else:
    print(1)

3


In [269]:
def ex():
    a = [(3, 'd'), (10, '6')]
    return a[0]

b, c = ex()
b

3

In [238]:
parameters = [{'n_neighbors': [1, 2, 3, 4, 5, 10, 15, 20], 'p': [1, 2]}]
grid = GridSearchCV(KNeighborsRegressor(), parameters)
grid.fit(X_train, y_train)
grid.best_params_['p']

1

In [249]:
grid.best_params_

{'n_neighbors': 3, 'p': 1}

In [257]:
list(grid.best_params_.keys())

['n_neighbors', 'p']

In [366]:
tree = DecisionTreeRegressor(max_depth=None, random_state=3)

In [367]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=3)

In [368]:
tree.score(X_test, y_test)

0.9448573743695495

In [369]:
reg = LinearRegression()

In [370]:
reg.fit(X_train, y_train)

LinearRegression()

In [371]:
reg.score(X_test, y_test)

0.8879353446048815

In [338]:
from sklearn.metrics import r2_score

In [190]:
r2_score(y_test, knn.predict(X_test))

0.9491092778381445